https://colab.research.google.com/github/sozzzzhot/Test/blob/main/Stable_Diffusion_WebUi.ipynb

In [ ]:
%cd /content

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
!pip install -q xformers==0.0.20 triton==2.0.0 -U

!git clone -b v2.4 https://github.com/camenduru/ui
!git clone https://huggingface.co/embed/negative /content/ui/embeddings/negative
#!git clone https://huggingface.co/embed/lora /content/ui/models/Lora/positive
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/ui/models/ESRGAN -o 4x-UltraSharp.pth
#!git clone https://github.com/camenduru/sd-civitai-browser /content/ui/extensions/sd-civitai-browser
!git clone https://github.com/camenduru/control /content/ui/extensions/control
#!git clone https://github.com/fkunn1326/openpose-editor /content/ui/extensions/openpose-editor
!git clone https://github.com/camenduru/tunnels /content/ui/extensions/tunnels
%cd /content/ui
!git reset --hard
!git -C /content/ui/repositories/stable-diffusion-stability-ai reset --hard


# Variables
model_path = "/content/ui/models/Stable-diffusion"
ext_path = "/content/ui/extensions"
vae_path = "/content/ui/models/VAE"
lora_path = "/content/ui/models/Lora"
controlnet_path = "/content/ui/extensions/sd-webui-controlnet/models"
embd_path = "/content/ui/embeddings"
config_path = "/content/ui/config.json"

#@markdown #这个必填ModelLink
Model_Link = "https://huggingface.co/mnvvc/inpainting-lab/resolve/main/chilloutmix_NiPrunedFp32Fix-inpainting.inpainting.safetensors" #@param {type:"string"}

#@markdown #可选项，如果你想多加些模型用。
Model_Link2 = "" #@param {type:"string"}
Model_Link3 = "" #@param {type:"string"}
Model_Link4 = "" #@param {type:"string"}
#@markdown ##可选项，去下载lora网站，在下载按钮右键复制下载链接。第一个是示例。
#Lora
lora1 = "https://civitai.com/api/download/models/23250" #@param {type:"string"}
lora2 = "" #@param {type:"string"}
lora3 = "" #@param {type:"string"}
lora4 = "" #@param {type:"string"}
lora5 = "" #@param {type:"string"}
lora6 = "" #@param {type:"string"}
lora7 = "" #@param {type:"string"}
lora8 = "" #@param {type:"string"}
lora9 = "" #@param {type:"string"}
lora10 = "" #@param {type:"string"}
if lora1 != '':
  !wget $lora1 -P $lora_path --content-disposition
if lora2 != '':
  !wget $lora2 -P $lora_path --content-disposition
if lora3 != '':
  !wget $lora3 -P $lora_path --content-disposition
if lora4 != '':
  !wget $lora4 -P $lora_path --content-disposition
if lora5 != '':
  !wget $lora5 -P $lora_path --content-disposition
if lora6 != '':
  !wget $lora6 -P $lora_path --content-disposition
if lora7 != '':
  !wget $lora7 -P $lora_path --content-disposition
if lora8 != '':
  !wget $lora8 -P $lora_path --content-disposition
if lora9 != '':
  !wget $lora9 -P $lora_path --content-disposition
if lora10 != '':
  !wget $lora10 -P $lora_path --content-disposition

if Model_Link != "":
    !wget $Model_Link -P $model_path --content-disposition
if Model_Link2 != "":
    !wget $Model_Link2 -P $model_path --content-disposition
if Model_Link3 != "":
    !wget $Model_Link3 -P $model_path --content-disposition
if Model_Link4 != "":
    !wget $Model_Link4 -P $model_path --content-disposition

!wget https://raw.githubusercontent.com/sozzzzhot/sdzzzzz-cocolab/main/ui.js -O javascript/ui.js
!wget https://raw.githubusercontent.com/sozzzzhot/sdzzzzz-cocolab/main/ui.py -O modules/ui.py
!wget 'https://raw.githubusercontent.com/hanamizuki-ai/stable-diffusion-webui-localization-zh_Hans/master/localizations/zh-Hans%20(Stable).json' -O localizations/zh-Hans.json
!git clone https://github.com/thomasasfk/sd-webui-aspect-ratio-helper $ext_path/sd-webui-aspect-ratio-helper
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd14/resolve/main/sd-v1-4.ckpt -d /content/ui/models/Stable-diffusion -o sd-v1-4.ckpt

!sed -i -e '''/from modules import launch_utils/a\import os''' /content/ui/launch.py
!sed -i -e '''/        prepare_environment()/a\        os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/ui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/ui/launch.py
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /content/ui/modules/shared.py

!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple

In [ ]:
!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple